In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils

from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles

from keras import backend as K
import matplotlib.pyplot as plt
import re
import hexdump

print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.1.1


In [9]:
# load model
savedModel = load_model("saved_model/model")
savedModel.summary()

2021-12-30 19:19:25.770147: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-30 19:19:25.770196: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-30 19:19:25.770225: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2021-12-30 19:19:25.770491: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-12-30 19:19:25.791220: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2699905000 Hz
2021-12-30 19:19:25.791565: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x555c3dbddbb0 initialized for platform Host (this does not guarantee that XLA 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                48        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [30]:
input = np.array([[3,2]])
savedModel.predict(input)

array([[0.70741206]], dtype=float32)

In [12]:
def port(model, optimize=False, variable_name='model_data', pretty_print=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimize:
        if isinstance(optimize, bool):
            optimizers = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
        else:
            optimizers = optimize
        converter.optimizations = optimizers
    tflite_model = converter.convert()
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])

    return c_array.split(",")

In [14]:
list_data = port(savedModel)

2021-12-30 19:22:05.832125: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-12-30 19:22:05.832433: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-12-30 19:22:05.843420: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:814] Optimization results for grappler item: graph_to_optimize
2021-12-30 19:22:05.843463: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:816]   function_optimizer: Graph size after: 65 nodes (54), 103 edges (90), time = 3.065ms.
2021-12-30 19:22:05.843473: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:816]   function_optimizer: function_optimizer did nothing. time = 0.066ms.
2021-12-30 19:22:05.927057: I tensorflow/core/grappler/devices.cc:55] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-12-30 19:22:05.927223: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-12-30 19:22:0

In [26]:
#generar indices
n = 10
n_bytes = len(list_data)
print(n_bytes)

indices = []
for x in range(n,n_bytes,n):
    indices.append((x-n,x))
if (x<n_bytes):
    indices.append((x,n_bytes))

indices

file = open("model_v1.txt","w")

for indice1,indice2 in indices:
    text = ""
    for datos in list_data[indice1:indice2]:
        text+=datos.strip()+","
    text+="\n"
    file.write(text)

file.close()

2776


In [29]:
file = open("model_v2.txt","w")

for dato in list_data:

    file.write("{}\n".format(dato.strip()))

file.close()